# Azure ML Pipelines

Цель лабораторной работы: 

- публикация __Конвейеров машинного обучения__ (ML Pipelines) в Azure ML
- планирование выполнения Конвейеров машинного обучения.

## Подготовка среды

Импорт необходимых модулей и проверка версии AzureML SDK:

In [ ]:
import azureml.core
from azureml.core import Workspace

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Зададим параметры Эксперимента:

In [ ]:
experiment_name = 'ml_pipeline_demo'

experiment_dir = 'ml-pipeline-demo'
os.makedirs(experiment_dir, exist_ok=True)

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [ ]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

## Публикация ML Конвейера

После того как Вы создали ML Конвейер, Вы можете опубликовать его в качестве REST-сервиса. 
Вы запустили ML Конвейер в [предыдущей лаборатории](06A-ml-pipelines.ipynb). Используйте ссылку на этот запуск для публикации ML Конвейера.

In [ ]:
# Get the most recent run of the pipeline
pipeline_experiment = ws.experiments.get(experiment_name)
pipeline_run = list(pipeline_experiment.get_runs())[0]

print(f'{pipeline_run}')

In [ ]:
# Publish the pipeline from the run
pipeline = pipeline_run.publish_pipeline(name='ml_pipeline_demo',
                                         description='Demo ML Pipeline',
                                         version='1.0')

pipeline

Для опубликованного ML Конвейера была создана Конечная точка (Endpoint), которую также можно увидеть на странице Конечные точки (на вкладке Конечные точки конвейера) на портале Azure ML. 
URI Конечной точки также доступно как свойство опубликованного ML Конвейера:

In [ ]:
print(f'{pipeline.name} Endpoint URI: {pipeline.endpoint}')

## Использование Конечной точки ML Конвейера

Чтобы использовать Конечную точку, нужно выполнить вызов REST по протоколу HTTP. Этот запрос должен быть аутентифицирован, поэтому требуется заголовок авторизации (authentication header). Получим это заголовок:

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

auth = InteractiveLoginAuthentication()
auth_header = auth.get_authentication_header()

print("Authentication Header is ready.")

Запустим выполнение ML Конвейера: 

In [ ]:
import requests

response = requests.post(pipeline.endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})

run_id = response.json()["Id"]
print(f'Run id: {run_id}')

Идентификатор запуска Вам вернулся мнгновенно, но это не означает, что ML Конвейер уже выполнился. 

Посмотрите через портал Azure ML за статусом запуска ML Конвейера или с помощью следущего кода:

In [ ]:
from azureml.pipeline.core.run import PipelineRun

pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
pipeline_run.wait_for_completion(show_output=True)

## Планирование выполнения запусков ML Конвейера

Предположим, что клиника для больных сахарным диабетом собирает новые данные каждую неделю и добавляет их в набор данных. Вы можете запускать конвейер каждую неделю, чтобы переобучить модель с новыми данными.

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")

weekly_schedule = Schedule.create(ws, 
                                  name="weekly-diabetes-model-training", 
                                  description="Weekly Diabetes Model Training Pipeline",
                                  pipeline_id=pipeline.id, 
                                  experiment_name=experiment_name, 
                                  recurrence=recurrence)

print('Pipeline was scheduled.')

Получим список запланированных запусков:

In [ ]:
Schedule.list(ws)

## Вывод

## Полезные ссылки
